In [ ]:
!pip install transformers torch sounddevice numpy soundfile streamlit
!pip install librosa
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install datasets



In [ ]:
import streamlit as st
import soundfile as sf
from transformers import pipeline
import numpy as np
import tempfile
import os
import librosa
import time
from jiwer import wer

In [ ]:
import streamlit as st
from transformers import pipeline

# Load the ASR pipeline with caching and GPU support
@st.cache_resource
def load_pipeline():
    model_name = "/home/adhippc/miniconda3/envs/Fuse_ASR/codes/model"  # Update the path to your local model
    whisper_pipeline = pipeline("automatic-speech-recognition", model=model_name, device=0)  # Use GPU (device=0)
    return whisper_pipeline

# Initialize the pipeline
whisper_pipeline = load_pipeline()
st.title("Voice-to-Text Converter (Nepali)")


In [4]:
text_folder = 'paragraphs'
audio_folder = 'audios'

total_wer = 0
total_latency = 0
num_files = 100



In [ ]:



total_wer = 0
total_latency = 0
chunk_duration = 30

# Loop through audio and text files from 100 to 200
for i in range(100, 201):
    audio_file = os.path.join(audio_folder, f'audio_{i}.mp3')
    text_file = os.path.join(text_folder, f'paragraph_{i}.txt')

    # Load audio and text
    with open(text_file, 'r') as f:
        reference_text = f.read().strip()

    # Load audio file
    audio_data, sr = librosa.load(audio_file, sr=16000)

    # Split audio into chunks
    num_chunks = int(len(audio_data) / (sr * chunk_duration)) + 1

    chunk_predictions = []
    for j in range(num_chunks):
        start_sample = j * sr * chunk_duration
        end_sample = min((j + 1) * sr * chunk_duration, len(audio_data))
        audio_chunk = audio_data[start_sample:end_sample]

        # Measure latency and transcribe audio chunk
        start_time = time.time()
        prediction = whisper_pipeline(audio_chunk)["text"]
        latency = time.time() - start_time
        chunk_predictions.append(prediction)

    # Combine predictions from all chunks
    full_prediction = " ".join(chunk_predictions)

    # Calculate WER
    wer_value = wer(reference_text, full_prediction)

    # Accumulate WER and latency
    total_wer += wer_value
    total_latency += latency

    print(f'Audio {i}: WER = {wer_value:.4f}, Latency = {latency:.4f} seconds')

# Optionally, you can print the total WER and latency after the loop
average_wer = total_wer / (101)  # 101 because it goes from 100 to 200
average_latency = total_latency / (101)

print(f'Total WER: {total_wer:.4f}, Average WER: {average_wer:.4f}')
print(f'Total Latency: {total_latency:.4f} seconds, Average Latency: {average_latency:.4f} seconds')

